In [7]:
from os import getenv
import pandas as pd
from requests import get

In [8]:
API = getenv('AV_API')

In [9]:
url = f'https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_WEEKLY&symbol=BTC&market=USD&apikey={API}&datatype=csv'
response = get(url, timeout = 10)
print(response)

<Response [200]>


In [10]:
with open('data.csv', 'wb') as dataFile:
    dataFile.write(response.content)

In [11]:
data = pd.read_csv('data.csv')
data

,timestamp,open (USD),high (USD),low (USD),close (USD),open (USD).1,high (USD).1,low (USD).1,close (USD).1,volume,market cap (USD)
0,2023-05-01,29233.20,29320.00,29215.93,29292.94,29233.20,29320.00,29215.93,29292.94,1281.336740,1281.336740
1,2023-04-30,27590.59,30036.00,26942.82,29233.21,27590.59,30036.00,26942.82,29233.21,444613.477010,444613.477010
2,2023-04-23,30304.66,30485.00,27125.00,27590.60,30304.66,30485.00,27125.00,27590.60,430421.846460,430421.846460
3,2023-04-16,28323.76,31000.00,28170.00,30304.65,28323.76,31000.00,28170.00,30304.65,377573.740440,377573.740440
4,2023-04-09,28171.87,28775.00,27200.24,28323.76,28171.87,28775.00,27200.24,28323.76,306532.390140,306532.390140
...,...,...,...,...,...,...,...,...,...,...,...
138,2020-09-13,10255.89,10580.11,9850.00,10332.83,10255.89,10580.11,9850.00,10332.83,366759.994048,366759.994048
139,2020-09-06,11711.17,12050.85,9825.00,10256.20,11711.17,12050.85,9825.00,10256.20,581777.756525,581777.756525
140,2020-08-30,11648.12,11824.90,11117.64,11711.16,11648.12,11824.90,11117.64,11711.16,355153.169372,355153.169372
141,2020-08-23,11910.99,12468.00,11376.81,11648.13,11910.99,12468.00,11376.81,11648.13,426710.483502,426710.483502
